In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import flwr
import torch
import torch.nn as nn
import torch.nn.functional as F
from flwr.common import Context
from flwr.simulation import run_simulation
from torchvision import datasets, transforms

from fl_g13 import dataset as dataset_handler
from fl_g13.config import RAW_DATA_DIR


2025-05-06 11:24:48.766 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Load data

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])
cifar100_train = datasets.CIFAR100(root=RAW_DATA_DIR, train=True, download=True, transform=transform)
cifar100_test = datasets.CIFAR100(root=RAW_DATA_DIR, train=False, download=True, transform=transform)

In [6]:
### train val split
cifar100_test, _ = dataset_handler.train_test_split(cifar100_test, train_ratio=0.2)

### train val split
train_dataset, val_dataset = dataset_handler.train_test_split(cifar100_train, train_ratio=0.8)
# I.I.D Sharding Split
## k client
k = 100
clients_dataset_train = dataset_handler.iid_sharding(train_dataset, k)
clients_dataset_val = dataset_handler.iid_sharding(val_dataset, k)

## Tiny model

In [7]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # -> [B, 16, 32, 32]
        x = F.max_pool2d(x, 2)  # -> [B, 16, 16, 16]
        x = F.relu(self.conv2(x))  # -> [B, 32, 16, 16]
        x = F.max_pool2d(x, 2)  # -> [B, 32, 8, 8]
        x = x.view(x.size(0), -1)  # -> [B, 32*8*8]
        x = self.fc1(x)  # -> [B, 100]
        return x

## Init model , optimizer and loss function

In [8]:
from fl_g13.editing import SparseSGDM

BATCH_SIZE = 128
LR = 1e-3
model = TinyCNN().to(DEVICE)
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.AdamW(net.parameters(), lr=1e-4, weight_decay=0.04)
mask = [torch.ones_like(p, device=p.device) for p in model.parameters()]
optimizer = SparseSGDM(
    model.parameters(),
    mask=mask,
    lr=LR,
    momentum=0.9,
    weight_decay=1e-5
)
criterion = torch.nn.CrossEntropyLoss()

# Define the ClientApp

## Build module local

Build module local such that ClientApp can use it

In [16]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fl_g13 (pyproject.toml): started
  Building editable for fl_g13 (pyproject.toml): finished with status 'done'
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4649 sha256=a588eeccbccdbea56dbfe92e55bd612babd81e07366f39e361ccc99a5ffdec08
  Stored in directory: C:\Users\ADMIN\AppData\Local\Temp\pip-ephem-wheel-cache-w89n9yil\wheels\b7


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## create FlowerClient instances  

### Create instant of ClientApp

In [17]:
from fl_g13.fl_pytorch.client_app import get_client_app


def load_data_client(context: Context, **kwargs):
    partition_id = context.node_config["partition-id"]
    print(f"Client {partition_id} is ready to train")
    trainloader = DataLoader(clients_dataset_train[partition_id])
    valloader = DataLoader(clients_dataset_val[partition_id])
    return trainloader, valloader

mask_type='local'
local_epoches = 2
client = get_client_app(load_data_fn=load_data_client, model=model,
                        optimizer=optimizer, criterion=criterion, device=DEVICE,
                        local_epochs=local_epoches,
                        model_editing=True,
                        mask_type=mask_type,
                        sparsity=0.2
                        )

# Define the Flower ServerApp

Customize built-in strategy Federated Averaging (FedAvg) of Flower to combine hyperparams in server-side and save model for each k epoch

The strategy could also incremental training an

## Create instant of ServerApp

In [18]:

from pathlib import Path
from torch.utils.data import DataLoader
from fl_g13.fl_pytorch.server_app import get_server_app


def get_datatest_fn(context: Context):
    return DataLoader(cifar100_test)


## checkpoints directory
current_path = Path.cwd()
model_test_path = current_path / "../models/fl/model_editing_test"
model_test_path.resolve()

num_rounds = 2
save_every = 1
fraction_fit = 1.0  # Sample 100% of available clients for training
fraction_evaluate = 0.5  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = DEVICE
use_wandb = False

server = get_server_app(checkpoint_dir=model_test_path.resolve(),
                        model_class=TinyCNN,
                        optimizer=optimizer, criterion=criterion, get_datatest_fn=get_datatest_fn,
                        num_rounds=num_rounds,
                        fraction_fit=fraction_fit,
                        fraction_evaluate=fraction_evaluate,
                        min_fit_clients=min_fit_clients,
                        min_evaluate_clients=min_evaluate_clients,
                        min_available_clients=min_available_clients,
                        device=device,
                        use_wandb=use_wandb,
                        save_every=save_every
                        )

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl\model_editing_test\FL_TinyCNN_epoch_5.pth
📦 Model class in checkpoint: TinyCNN
✅ Loaded checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl\model_editing_test\FL_TinyCNN_epoch_5.pth, resuming at epoch 6


# Run the training


In [19]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 0.25}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [20]:
NUM_CLIENTS = 10

In [21]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Continue train model from epoch 6


Eval progress: 100%|██████████| 2000/2000 [00:02<00:00, 740.80batch/s]
INFO :      ROUND 0💡 New best global model found: 0.053500
INFO :      initial parameters (loss, other metrics): 4.225203928768635, {'centralized_accuracy': 0.0535}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


Server round 0 - loss: 4.225203928768635, metrics: {'centralized_accuracy': 0.0535}


(ClientAppActor pid=20536) 2025-05-06 11:34:05.547 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13
Fisher Score:   0%|          | 0/100 [00:00<?, ?batch/s]


...

Eval progress: 100%|██████████| 100/100 [00:00<00:00, 700.27batch/s]


(ClientAppActor pid=1684) 🚀 Epoch 2/2 (100.00%) Completed
(ClientAppActor pid=1684) 	📊 Training Loss: 3.8645
(ClientAppActor pid=1684) 	✅ Training Accuracy: 8.75%
(ClientAppActor pid=1684) 	⏳ Elapsed Time: 1.39s | ETA: 0.00s
(ClientAppActor pid=1684) 	🕒 Completed At: 11:35


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 117.36s
INFO :      	History (loss, distributed):
INFO :      		round 1: 4.354769141435623
INFO :      		round 2: 4.370063863396645
INFO :      	History (loss, centralized):
INFO :      		round 0: 4.225203928768635
INFO :      		round 1: 4.172763308942318
INFO :      		round 2: 4.12708351546526
INFO :      	History (metrics, distributed, fit):
INFO :      	{'avg_drift': [(1, 2.0218725323677065), (2, 2.190437364578247)],
INFO :      	 'avg_train_loss': [(1, 8.136771921578795), (2, 7.95445452123927)]}
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'federated_evaluate_accuracy': [(1, 0.052), (2, 0.05)]}
INFO :      	History (metrics, centralized):
INFO :      	{'centralized_accuracy': [(0, 0.0535), (1, 0.0635), (2, 0.076)]}
INFO :      
Eval progress: 100%|██████████| 100/100 [00:00<00:00, 781.04batch/s]


(ClientAppActor pid=20536) Client 9 is ready to train [repeated 4x across cluster]


Eval progress: 100%|██████████| 100/100 [00:00<00:00, 660.86batch/s] [repeated 3x across cluster]
